In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
# Chemins d'accès aux dossiers d'images
train_folder = './datasetTraiter/TrainData/'
test_folder = './datasetTraiter/test/'

# Fonction pour extraire les caractéristiques HOG d'une image redimensionnée
def extract_hog_features_resized(image, target_size=(128, 128)):
    # Redimensionner l'image à la taille cible
    resized_image = cv2.resize(image, target_size)
    
    # Convertir l'image redimensionnée en niveaux de gris si elle a plus de deux dimensions
    if len(resized_image.shape) > 2:
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    
    # Extraire les caractéristiques HOG de l'image redimensionnée
    features = hog(resized_image, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=False,
                transform_sqrt=True, block_norm='L2-Hys')
    return features


# Fonction pour charger les noms de fichiers à partir des fichiers Excel
def load_image_names(excel_file):
    return pd.read_excel(excel_file)['image_name'].tolist()

# Charger les noms des images saines et malades
healthy_images = load_image_names('./datasetTraiter/healthy_images.xlsx')
maladie_images = load_image_names('./datasetTraiter/maladie_images.xlsx')

# Charger les noms des images de test
test_images = load_image_names('./datasetTraiter/test_images.xlsx')

# Extraire les caractéristiques HOG pour les images d'entraînement
# Utilisation de la nouvelle fonction pour extraire les caractéristiques HOG des images d'entraînement
hog_features_train = []
train_labels = []
for image_name in healthy_images:
    image_path = os.path.join(train_folder, 'healthy', image_name)
    image = cv2.imread(image_path)
    if image is not None:
        hog_features = extract_hog_features_resized(image)
        hog_features_train.append(hog_features)
        train_labels.append(0)  # 0 pour healthy

print('done 1')


for image_name in maladie_images:
    image_path = os.path.join(train_folder, 'maladie', image_name)
    image = cv2.imread(image_path)
    if image is not None:
        hog_features = extract_hog_features_resized(image)
        hog_features_train.append(hog_features)
        train_labels.append(1)  # 1 pour maladie

print('done 2')


# Convertir les listes en tableaux numpy
X_train = np.array(hog_features_train)
y_train = np.array(train_labels)


# Utilisation de la nouvelle fonction pour extraire les caractéristiques HOG des images de test
hog_features_test = []
for image_name in test_images:
    image_path = os.path.join(test_folder, image_name)
    image = cv2.imread(image_path)
    if image is not None:
        hog_features = extract_hog_features_resized(image)
        hog_features_test.append(hog_features)



# Convertir la liste en tableau numpy
X_train = np.array(hog_features_train)
y_train = np.array(train_labels)
X_test = np.array(hog_features_test)

# Application de PCA sur les caractéristiques d'entraînement et de test
pca = PCA(n_components=0.90)
pca.fit(X_train)  # Fit seulement sur les données d'entraînement

print('done 3')

X_train_reduced = pca.transform(X_train)
X_test_reduced = pca.transform(X_test)

print('done 4')

# Normalisation des caractéristiques HOG
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reduced)
X_test_scaled = scaler.transform(X_test_reduced)

print('done 5')

# Affichage des dimensions après PCA et normalisation
print("Dimensions des caractéristiques HOG réduites et normalisées pour l'ensemble d'entraînement :", X_train_scaled.shape)
print("Dimensions des caractéristiques HOG réduites et normalisées pour l'ensemble de test :", X_test_scaled.shape)


done 1
done 2
done 3
done 4
done 5
Dimensions des caractéristiques HOG réduites et normalisées pour l'ensemble d'entraînement : (1821, 883)
Dimensions des caractéristiques HOG réduites et normalisées pour l'ensemble de test : (1821, 883)


In [3]:
from sklearn.model_selection import cross_val_score

# Définir le modèle SVM
svm_model = SVC(kernel='rbf', C=9.74, gamma='scale')

# Calculer la précision du modèle en utilisant la validation croisée
# 'cv' détermine le nombre de plis dans la validation croisée
# 'scoring' spécifie la métrique d'évaluation (dans ce cas, nous utilisons 'accuracy')
# La fonction cross_val_score renvoie un tableau de scores pour chaque pli de validation croisée
scores = cross_val_score(svm_model, X_train_scaled, y_train, cv=5, scoring='accuracy')
# Initialiser le modèle SVM

# Entraîner le modèle SVM
svm_model.fit(X_train_scaled, y_train)

# Calculer la moyenne des scores de validation croisée pour obtenir une estimation de la performance du modèle
mean_accuracy = scores.mean()

print("Mean Accuracy (Cross-Validation):", mean_accuracy)


Mean Accuracy (Cross-Validation): 0.7287279843444228


In [9]:
from PIL import Image

# Chemin local de l'image à prédire
# image_path = "../foliar_diseases_apple_trees_project/dataset/datasetTraiter/TrainData/healthy/Train_151.jpg"
image_path = "./datasetTraiter/TrainData/healthy/Train_2.jpg"


# Charger l'image à partir du chemin local
image = Image.open(image_path)

# Redimensionner l'image si nécessaire et extraire les caractéristiques HOG
image_resized = image.resize((64, 64))  # Redimensionner l'image à la taille attendue par le modèle
hog_features = extract_hog_features_resized(np.array(image_resized))  # Extraire les caractéristiques HOG de l'image redimensionnée

# Normaliser les caractéristiques HOG
hog_features_scaled = scaler.transform(pca.transform([hog_features]))

# Faire une prédiction avec le modèle SVM
prediction = svm_model.predict(hog_features_scaled)

print("")
# Afficher la prédiction
if prediction == 0:
    print("La feuille n'est pas malade.")
else:
    print("La feuille est malade.")


La feuille n'est pas malade.


In [10]:
from PIL import Image

# Chemin local de l'image à prédire
# image_path = "../foliar_diseases_apple_trees_project/dataset/datasetTraiter/TrainData/healthy/Train_151.jpg"
image_path = "./datasetTraiter/TrainData/maladie/Train_3.jpg"


# Charger l'image à partir du chemin local
image = Image.open(image_path)

# Redimensionner l'image si nécessaire et extraire les caractéristiques HOG
image_resized = image.resize((64, 64))  # Redimensionner l'image à la taille attendue par le modèle
hog_features = extract_hog_features_resized(np.array(image_resized))  # Extraire les caractéristiques HOG de l'image redimensionnée

# Normaliser les caractéristiques HOG
hog_features_scaled = scaler.transform(pca.transform([hog_features]))

# Faire une prédiction avec le modèle SVM
prediction = svm_model.predict(hog_features_scaled)

print("")
# Afficher la prédiction
if prediction == 0:
    print("La feuille n'est pas malade.")
else:
    print("La feuille est malade.")


La feuille est malade.


In [6]:
import joblib

# Serialize the trained model
# joblib.dump(svm_model, 'svm_model.pkl')
# joblib.dump(scaler, 'scaler_model.pkl')
# joblib.dump(pca, 'pca_model.pkl')